In [1]:
%load_ext autoreload

In [2]:
import numpy as np
from functools import partial
import visualizations_utils as viz_utils
from iris_plant_visualizer import IrisPlantVisualizer
import ipywidgets as widgets
from IPython.display import display
from scipy.linalg import block_diag
import matplotlib.pyplot as plt
from pathlib import Path
import os

In [3]:
#pydrake imports
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.geometry import Role, GeometrySet, CollisionFilterDeclaration
from pydrake.solvers import mathematicalprogram as mp
from pydrake.all import RigidTransform, RollPitchYaw, RevoluteJoint
from pydrake.all import RotationMatrix
import pydrake.symbolic as sym
import time
from pydrake.all import CspaceFreeLine, CspaceFreeRegion
import pydrake.multibody.rational_forward_kinematics as rational_forward_kinematics
from pydrake.all import VerificationOption

from pydrake.all import RationalForwardKinematics
from pydrake.geometry.optimization import IrisOptionsRationalSpace, IrisInRationalConfigurationSpace, HPolyhedron, Hyperellipsoid

In [4]:
import logging
drake_logger = logging.getLogger("drake")
# drake_logger.setLevel(logging.DEBUG)

# Build and set up the visualization the plant and the visualization of the C-space obstacle

Note that running this cell multiple times will establish multiple meshcat instances which can fill up your memory. It is a good idea to call "pkill -f meshcat" from the command line before re-running this cell


In [5]:
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
model_file = FindResourceOrThrow("drake/manipulation/models/iiwa_description/iiwa7/iiwa7_with_box_collision.sdf")
box_file = FindResourceOrThrow("drake/C_Iris_Examples/assets/shelves.sdf")


parser = Parser(plant)


models = []
models.append(Parser(plant, scene_graph).AddModelFromFile(model_file))
models.append(Parser(plant, scene_graph).AddModelFromFile(box_file))

sp = 0.4
x_fac = 1.2
locs = [ [0,0,0], 
        [x_fac*sp, 1.4*sp,0.4], [x_fac*sp,-1.4*sp,0.4], [-x_fac*sp,-1.4*sp,0.4], [-x_fac*sp,1.4*sp,0.4], 
        [0.0 ,0 , 0.95], [0.0 ,0 , -0.05]] 
idx = 0
for model in models:
    plant.WeldFrames(plant.world_frame(), plant.get_body(plant.GetBodyIndices(model)[0]).body_frame(),
                     RigidTransform(locs[idx]))
    idx +=1


plant.Finalize()


        
# construct the RationalForwardKinematics of this plant. This object handles the
# computations for the forward kinematics in the tangent-configuration space
Ratfk = RationalForwardKinematics(plant)

# the point about which we will take the stereographic projections
q_star = np.zeros_like(plant.GetPositionLowerLimits())

#compute limits in t-space
limits_t = []
for q in [plant.GetPositionLowerLimits(), plant.GetPositionUpperLimits()]:
    limits_t.append(np.clip(Ratfk.ComputeTValue(np.array(q), q_star), -3, 3)/100)

do_viz = True

# This line builds the visualization. Change the viz_role to Role.kIllustration if you
# want to see the plant with its illustrated geometry or to Role.kProximity
visualizer = IrisPlantVisualizer(plant, builder, scene_graph, viz_role=Role.kIllustration)
diagram = visualizer.diagram

# This line will run marching cubes to generate a mesh of the C-space obstacle
# Increase N to increase the resolution of the C-space obstacle.


INFO:drake:Meshcat listening for connections at http://localhost:7000
INFO:drake:Meshcat listening for connections at http://localhost:7001
/home/amice/Documents/coding_projects/drake/C_Iris_Examples/iris_plant_visualizer.py:31: DrakeDeprecationWarning: MeshcatVisualizerCpp has been renamed to MeshcatVisualizer. The deprecated code will be removed from Drake on or after 2022-11-01.
  self.vis = MeshcatVisualizerCpp.AddToBuilder(builder, scene_graph, self.meshcat1)


In [6]:
visualizer.meshcat2.Delete("/prm")

In [7]:
# draw prm
import prm
from pydrake.all import Rgba

# limits = [np.array(t_low), np.array(q_high)]

visualizer.meshcat2.Delete("/prm")

def plot_prm(nodes, adjacency_list, width, color = Rgba(0.0, 0.0, 1, 1), prefix = ""):
    plt_idx = 0
    for node_idx in range(nodes.shape[0]):
        pos1 = np.append(nodes[node_idx, :],0)
        for edge_idx in range(len(adjacency_list[node_idx])): 
            pos2 = np.append(nodes[adjacency_list[node_idx][edge_idx], :],0)
            name = f"/{prefix}prm/rm/line{plt_idx}"
            visualizer.meshcat2.SetLine(name,  np.hstack([pos1[:,np.newaxis], pos2[:, np.newaxis]]),
                                 line_width = width, rgba = color)
            plt_idx +=1
            
plotting_fn_handle = partial(plot_prm, width = 0.01)

def collision(pos, col_func_handle):
    return col_func_handle(pos[0], pos[1])

def collision_bad(pos, col_func_handle):
    return 1-col_func_handle(pos[0], pos[1])

prm_col_fn_handle = partial(collision, col_func_handle = visualizer.col_func_handle_rational)
prm_col_fn_handle_bad = partial(collision_bad, col_func_handle = visualizer.col_func_handle_rational)

PRM = prm.PRM( 
            limits_t,
            num_points = 20,
            col_func_handle = prm_col_fn_handle,
            num_neighbours = 5, 
            dist_thresh = .1,
            num_col_checks = 10,
            max_it = 1e5,
            verbose = True,
            plotcallback = None
            )

# PRM_bad = prm.PRM( 
#             limits_t,
#             num_points = 2000,
#             col_func_handle = prm_col_fn_handle_bad,
#             num_neighbours = 5, 
#             dist_thresh = .5,
#             num_col_checks = 10,
#             verbose = True,
#             plotcallback = None
#             )

# PRM.add_start_end(start, target)
PRM.plot()
tot_num_edges = len(PRM.adjacency_list)* len(PRM.adjacency_list[0])
# path, sp_length = PRM.find_shortest_path()

# mat = meshcat.geometry.MeshLambertMaterial(color= 0xFFF812 , wireframe=True)
# mat.wireframeLinewidth = 2.0
# num_waypoints = len(path)
# for idx in range(num_waypoints-1):
#     vis2['prm']['path']['path' + str(idx)].set_object( meshcat_line(path[idx], path[idx+1],width = 0.01), mat)
# traj= utils.PWLinTraj(path, 5.0)

[PRM] Samples 0
[PRM] Nodes connected: 0


In [8]:
def make_prm_line_pairs(PRM):
    endpoint_index_set = set()
    for neighbors in PRM.adjacency_list:
        for n in neighbors[1:]:
            endpoint_index_set.add((neighbors[0], n))
    s0 = np.zeros((len(endpoint_index_set), plant.num_positions()))
    s1 = np.zeros((len(endpoint_index_set), plant.num_positions()))
    for i, (idx0, idx1) in enumerate(endpoint_index_set):
        s0[i] = PRM.nodes[idx0]
        s1[i] = PRM.nodes[idx1]
    return s0, s1
s0_mat_safe, s1_mat_safe = make_prm_line_pairs(PRM)
# s0_mat_unsafe, s1_mat_unsafe = make_prm_line_pairs(PRM_bad)
print(s0_mat_safe.shape)
# print(s0_mat_unsafe.shape)

(80, 7)


In [9]:
PRM.adjacency_list

[[0, 15, 12, 14, 13],
 [1, 9, 3, 11, 18],
 [2, 17, 3, 16, 19],
 [3, 9, 1, 6, 8],
 [4, 5, 8, 11, 3],
 [5, 8, 4, 3, 13],
 [6, 10, 9, 12, 11],
 [7, 10, 14, 9, 13],
 [8, 5, 14, 12, 4],
 [9, 3, 10, 6, 11],
 [10, 7, 14, 6, 9],
 [11, 9, 1, 4, 6],
 [12, 14, 18, 15, 0],
 [13, 15, 14, 10, 0],
 [14, 7, 12, 10, 8],
 [15, 0, 12, 13, 14],
 [16, 6, 15, 17, 2],
 [17, 12, 2, 3, 16],
 [18, 12, 15, 1, 9],
 [19, 1, 3, 9, 6]]

## Set up the sliders so we can move the plant around manually

In [10]:
sliders = []
for i in range(plant.num_positions()):
    sliders.append(widgets.FloatSlider(min=plant.GetPositionLowerLimits()[i],
                                       max=plant.GetPositionUpperLimits()[i], 
                                       value=0, description=f'q{i}'))
# sliders.append(widgets.FloatSlider(min=q_low[2], max=q_high[2], value=0, description='q2'))

q = np.zeros(plant.num_positions())
def handle_slider_change(change, idx):
    q[idx] = change['new']
    visualizer.showres(q)
    visualizer.visualize_planes()
    
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1

for slider in sliders:
    display(slider)


# visualizer.jupyter_cell()

Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.


Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.


Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.


Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.


Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.


Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.


Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.


In [11]:
# filter fused joints self collisions so they don't interfere with collision engine
digaram = visualizer.diagram
context = visualizer.diagram_context
sg_context = scene_graph.GetMyContextFromRoot(context)
inspector = scene_graph.model_inspector()

pairs = scene_graph.get_query_output_port().Eval(sg_context).inspector().GetCollisionCandidates()
print(f"Number of pairs {len(pairs)}")

# gids = [gid for gid in inspector.GetGeometryIds(GeometrySet(inspector.GetAllGeometryIds()), Role.kProximity)]
# get_name_of_gid = lambda gid : inspector.GetName(gid)
# gids.sort(key=get_name_of_gid)
# right_sweeper_gids = [gid for gid in gids if "right_sweeper::" in get_name_of_gid(gid)]
# left_sweeper_gids = [gid for gid in gids if "left_sweeper::" in get_name_of_gid(gid)]

# right_sweeper_fused_col_geom = right_sweeper_gids[2:]
# right_sweeper_fused_set = GeometrySet(right_sweeper_fused_col_geom)
# left_sweeper_fused_col_geom = left_sweeper_gids[4:]
# left_sweeper_fused_set = GeometrySet(left_sweeper_fused_col_geom)
# scene_graph.collision_filter_manager()\
#             .Apply(CollisionFilterDeclaration().ExcludeWithin(right_sweeper_fused_set))
# scene_graph.collision_filter_manager()\
#             .Apply(CollisionFilterDeclaration().ExcludeWithin(left_sweeper_fused_set))

# right_sweeper_end_gid = right_sweeper_gids[-1]
# left_sweeper_end_gid = left_sweeper_gids[-1]
# id_pairs_of_interest = [(right_sweeper_end_gid, left_sweeper_end_gid),
#                        ]
# visualizer.collision_pairs_of_interest = id_pairs_of_interest


Number of pairs 42


In [12]:
def count_num_safe(safe_points_vect):
    num_safe = 0
    for val in safe_points_vect:
        if(val):
            num_safe += 1
    return num_safe
def make_certifier_and_certify(use_affine = True):
    t0 = time.time()
    plane_type = rational_forward_kinematics.SeparatingPlaneOrder.kAffine if use_affine \
                    else rational_forward_kinematics.SeparatingPlaneOrder.kConstant
    line_certifier = CspaceFreeLine(diagram, plant, scene_graph,
                                    plane_type, 
                                    q_star,
                                    set(), VerificationOption()
                                   )
    t1 = time.time()
    print(f"Time to construct line certifier = {t1-t0}s")
    
    t0 = time.time()
    safe_points, planes = line_certifier.CertifyTangentConfigurationSpaceLines(s0_mat_safe, s1_mat_safe)
    t1 = time.time()
    print(f"Certification of safe PRM in {t1-t0}s")
    num_safe1 = count_num_safe(safe_points)
    print(f"Certified {num_safe1}/{len(safe_points)} as safe")
    print(f"Failed to certify {len(safe_points)-num_safe1}/{len(safe_points)} safe points as safe")
    
#     t0 = time.time()
#     unsafe_points, unsafe_planes = line_certifier.CertifyTangentConfigurationSpaceLines(s0_mat_unsafe, s1_mat_unsafe)
#     t1 = time.time()
#     print(f"Certification of unsafe PRM in {t1-t0}s")
#     num_safe2 = count_num_safe(safe_points)
#     print(f"Incorrectly certified {num_safe2}/{len(safe_points)} as safe")
#     print(f"Classified {len(safe_points)-num_safe2}/{len(safe_points)} unsafe points as unsafe")
    return (safe_points, planes, num_safe1), (unsafe_points, unsafe_planes, num_safe2)

In [ ]:
ret_safe_affine, ret_unsafe_affine = make_certifier_and_certify(use_affine = True)

INFO:drake:numerator poly constructed in 0 s. Has degree: 3
INFO:drake:numerator poly constructed in 0 s. Has degree: 3
INFO:drake:numerator poly constructed in 0 s. Has degree: 3
INFO:drake:numerator poly constructed in 0 s. Has degree: 3
INFO:drake:numerator poly constructed in 0 s. Has degree: 3
INFO:drake:numerator poly constructed in 0 s. Has degree: 3
INFO:drake:numerator poly constructed in 0 s. Has degree: 3
INFO:drake:numerator poly constructed in 0 s. Has degree: 3
INFO:drake:numerator poly constructed in 0 s. Has degree: 3
INFO:drake:numerator poly constructed in 0 s. Has degree: 3
INFO:drake:numerator poly constructed in 0 s. Has degree: 3
INFO:drake:numerator poly constructed in 0 s. Has degree: 3
INFO:drake:numerator poly constructed in 0 s. Has degree: 3
INFO:drake:numerator poly constructed in 0 s. Has degree: 3
INFO:drake:numerator poly constructed in 0 s. Has degree: 3
INFO:drake:numerator poly constructed in 0 s. Has degree: 3
INFO:drake:numerator poly constructed in

INFO:drake:numerator poly constructed in 0 s. Has degree: 5
INFO:drake:numerator poly constructed in 0 s. Has degree: 5
INFO:drake:numerator poly constructed in 0 s. Has degree: 5
INFO:drake:numerator poly constructed in 0 s. Has degree: 5
INFO:drake:numerator poly constructed in 0 s. Has degree: 5
INFO:drake:numerator poly constructed in 0 s. Has degree: 5
INFO:drake:numerator poly constructed in 0 s. Has degree: 5
INFO:drake:numerator poly constructed in 0 s. Has degree: 5
INFO:drake:numerator poly constructed in 0 s. Has degree: 5
INFO:drake:numerator poly constructed in 0 s. Has degree: 5
INFO:drake:numerator poly constructed in 0.007 s. Has degree: 7
INFO:drake:numerator poly constructed in 0.006 s. Has degree: 7
INFO:drake:numerator poly constructed in 0.005 s. Has degree: 7
INFO:drake:numerator poly constructed in 0.006 s. Has degree: 7
INFO:drake:numerator poly constructed in 0.006 s. Has degree: 7
INFO:drake:numerator poly constructed in 0.006 s. Has degree: 7
INFO:drake:numer

INFO:drake:numerator poly constructed in 0 s. Has degree: 5
INFO:drake:numerator poly constructed in 0 s. Has degree: 5
INFO:drake:numerator poly constructed in 0 s. Has degree: 5
INFO:drake:numerator poly constructed in 0.178 s. Has degree: 7
INFO:drake:numerator poly constructed in 0.006 s. Has degree: 7
INFO:drake:numerator poly constructed in 0.006 s. Has degree: 7
INFO:drake:numerator poly constructed in 0.005 s. Has degree: 7
INFO:drake:numerator poly constructed in 0.006 s. Has degree: 7
INFO:drake:numerator poly constructed in 0.005 s. Has degree: 7
INFO:drake:numerator poly constructed in 0.005 s. Has degree: 7
INFO:drake:numerator poly constructed in 0.005 s. Has degree: 7
INFO:drake:numerator poly constructed in 0.001 s. Has degree: 5
INFO:drake:numerator poly constructed in 0 s. Has degree: 5
INFO:drake:numerator poly constructed in 0 s. Has degree: 5
INFO:drake:numerator poly constructed in 0 s. Has degree: 5
INFO:drake:numerator poly constructed in 0 s. Has degree: 5
INFO

INFO:drake:numerator poly constructed in 0 s. Has degree: 5
INFO:drake:numerator poly constructed in 0 s. Has degree: 5
INFO:drake:numerator poly constructed in 0 s. Has degree: 5
INFO:drake:numerator poly constructed in 0 s. Has degree: 3
INFO:drake:numerator poly constructed in 0 s. Has degree: 3
INFO:drake:numerator poly constructed in 0 s. Has degree: 3
INFO:drake:numerator poly constructed in 0 s. Has degree: 3
INFO:drake:numerator poly constructed in 0 s. Has degree: 3
INFO:drake:numerator poly constructed in 0 s. Has degree: 3
INFO:drake:numerator poly constructed in 0 s. Has degree: 3
INFO:drake:numerator poly constructed in 0 s. Has degree: 3
INFO:drake:numerator poly constructed in 0 s. Has degree: 5
INFO:drake:numerator poly constructed in 0 s. Has degree: 5
INFO:drake:numerator poly constructed in 0 s. Has degree: 5
INFO:drake:numerator poly constructed in 0 s. Has degree: 5
INFO:drake:numerator poly constructed in 0 s. Has degree: 5
INFO:drake:numerator poly constructed in

INFO:drake:numerator poly constructed in 0.001 s. Has degree: 5
INFO:drake:numerator poly constructed in 0.001 s. Has degree: 5
INFO:drake:numerator poly constructed in 0.001 s. Has degree: 5
INFO:drake:numerator poly constructed in 0.007 s. Has degree: 7
INFO:drake:numerator poly constructed in 0.006 s. Has degree: 7
INFO:drake:numerator poly constructed in 0.006 s. Has degree: 7
INFO:drake:numerator poly constructed in 0.005 s. Has degree: 7
INFO:drake:numerator poly constructed in 0.005 s. Has degree: 7
INFO:drake:numerator poly constructed in 0.005 s. Has degree: 7
INFO:drake:numerator poly constructed in 0.005 s. Has degree: 7
INFO:drake:numerator poly constructed in 0.007 s. Has degree: 7
INFO:drake:numerator poly constructed in 0.001 s. Has degree: 5
INFO:drake:numerator poly constructed in 0.001 s. Has degree: 5
INFO:drake:numerator poly constructed in 0.001 s. Has degree: 5
INFO:drake:numerator poly constructed in 0 s. Has degree: 5
INFO:drake:numerator poly constructed in 0.0

INFO:drake:numerator poly constructed in 0.012 s. Has degree: 7
